In [4]:
!pip install transformers torch accelerate pandas openpyxl

In [5]:
from google.colab import files

files.upload()

Saving input_adjectives.xlsx to input_adjectives.xlsx
Saving input_sentences.xlsx to input_sentences.xlsx


{'input_adjectives.xlsx': b'PK\x03\x04\x14\x00\x06\x00\x08\x00\x00\x00!\x00\xdd+\x8bXl\x01\x00\x00\x10\x05\x00\x00\x13\x00\x08\x02[Content_Types].xml \xa2\x04\x02(\xa0\x00\x02\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\

In [6]:
!ls

input_adjectives.xlsx  input_sentences.xlsx  sample_data


In [7]:
import pandas as pd

In [8]:
adjectives_df = pd.read_excel("input_adjectives.xlsx")
sentences_df  = pd.read_excel("input_sentences.xlsx")

print(adjectives_df.head())
print(sentences_df.head())


         LEMMA
0  abbagliante
1   abbondante
2       acerbo
3        acido
4        acuto
         LEMMA POS/NEG LEX_ANT      DERIVAZIONE  \
0  abbagliante     NaN     NaN  deverbale_ppres   
1  abbagliante     NaN     NaN  deverbale_ppres   
2   abbondante     pos  scarso  deverbale_ppres   
3   abbondante     pos  scarso  deverbale_ppres   
4       acerbo     neg  maturo         semplice   

                                           SENTENCES CLASSIFICAZIONE  
0    Il risultato è, più che luminoso, abbagliante .             NaN  
1  Davanti a te appare una spiaggia spettacolare,...             NaN  
2  I lettini in spiaggia ed in piscina sono abbon...             NaN  
3  Dopo la posa sarà necessario un risciacquo con...             NaN  
4  I vini nuovi od acerbi sono più nocivi che utili.             NaN  


In [9]:
#Minerva upload + verify tokenization
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

MODEL_ID = "sapienzanlp/Minerva-7B-instruct-v1.0"

tokenizer = AutoTokenizer.from_pretrained(MODEL_ID)
model = AutoModelForCausalLM.from_pretrained(
    MODEL_ID,
    torch_dtype=torch.bfloat16,
    device_map="auto"
)
tokens = tokenizer.tokenize("anormale")
print(tokens)
token_ids = tokenizer.convert_tokens_to_ids(tokens)
print(token_ids)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/670 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/761 [00:00<?, ?B/s]

...

model-00002-of-00003.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.90G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/226 [00:00<?, ?B/s]

['Ġan', 'orm', 'ale']
[283, 1035, 399]


In [10]:
def generate(prompt, max_new_tokens=128):
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    outputs = model.generate(
        **inputs,
        max_new_tokens=max_new_tokens,
        do_sample=False,
        temperature=0.0
    )
    return tokenizer.decode(outputs[0], skip_special_tokens=True).strip()

Tentativo prompt 1

1) *Solo* *aggettivi*

In [15]:
def adjectives_prompt1(lemma: str) -> bool:
    prompt = f"""Dato l'aggettivo "{lemma}",
    se esiste una forma dell'aggettivo con uno dei seguenti prefissi:
    in-, im-, il-, ir-, dis-, a-, de-, s-, mis-,
    restituisci SOLO quella forma.

    Altrimenti, restituisci SOLO:
    NO.

    Nessuna spiegazione.
    """
    return generate(prompt)

In [16]:
adjectives_df["MINERVA_OUTPUT"] = adjectives_df["LEMMA"].apply(
    adjectives_prompt1
)


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for o

In [17]:
adjectives_df.to_excel("output_adjectives_1.xlsx", index=False)
files.download("output_adjectives_1.xlsx")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

2) *Aggettivi in contesto*

In [26]:
def sentences_prompt1(sentence, lemma):
    prompt = f"""Data la frase:

{sentence}

L'aggettivo "{lemma}" appare nella frase.

Se esiste una forma negativa dell'aggettivo con prefisso di negazione
(ad esempio: in-, im-, il-, ir-, dis-, a-, de-, s-, mis-),
riscrivi la frase usando quella forma.

Altrimenti, lascia la frase invariata.

Restituisci SOLO la frase modificata.
Nessuna spiegazione.
"""
    return generate(prompt, max_new_tokens=50)

In [28]:
sentences_df["MINERVA_OUTPUT"] = sentences_df.apply(
    lambda row: sentences_prompt1(row["SENTENCES"], row["LEMMA"]),
    axis=1
)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for o

In [29]:
sentences_df.to_excel("output_sentences_1.xlsx", index=False)
files.download("output_sentences_1.xlsx")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>